<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("ZAF_1.dta")
stata_df.head()

,period,period_start_day,value,frequency,dataset_code,dataset_name,freq,topic,unit,indexed_at,provider_code,series_code,series_name,series_num
0,1946,1946-01-01,NA,annual,QB,Quarterly Bulletin,A,FM,PERC,2022-04-04T17:36:18.525Z,SARB,KBP2000J,Yield on loan stock traded on the stock exchan...,Master
1,1947,1947-01-01,NA,annual,QB,Quarterly Bulletin,A,FM,PERC,2022-04-04T17:36:18.525Z,SARB,KBP2000J,Yield on loan stock traded on the stock exchan...,Master
2,1948,1948-01-01,NA,annual,QB,Quarterly Bulletin,A,FM,PERC,2022-04-04T17:36:18.525Z,SARB,KBP2000J,Yield on loan stock traded on the stock exchan...,Master
3,1949,1949-01-01,NA,annual,QB,Quarterly Bulletin,A,FM,PERC,2022-04-04T17:36:18.525Z,SARB,KBP2000J,Yield on loan stock traded on the stock exchan...,Master
4,1950,1950-01-01,NA,annual,QB,Quarterly Bulletin,A,FM,PERC,2022-04-04T17:36:18.525Z,SARB,KBP2000J,Yield on loan stock traded on the stock exchan...,Master


<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [2]:
python_df = pd.read_pickle("ZAF_1.pickle")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,topic,unit,freq,Topic,Unit,Frequency
0,annual,SARB,QB,Quarterly Bulletin,KBP1000J,South African Reserve Bank liabilities: Notes ...,1960,1960-01-01,255.0,255.0,MB,RMILL,A,Money and Banking,RMILL,Annual
1,annual,SARB,QB,Quarterly Bulletin,KBP1000J,South African Reserve Bank liabilities: Notes ...,1961,1961-01-01,276.0,276.0,MB,RMILL,A,Money and Banking,RMILL,Annual
2,annual,SARB,QB,Quarterly Bulletin,KBP1000J,South African Reserve Bank liabilities: Notes ...,1962,1962-01-01,302.0,302.0,MB,RMILL,A,Money and Banking,RMILL,Annual
3,annual,SARB,QB,Quarterly Bulletin,KBP1000J,South African Reserve Bank liabilities: Notes ...,1963,1963-01-01,337.0,337.0,MB,RMILL,A,Money and Banking,RMILL,Annual
4,annual,SARB,QB,Quarterly Bulletin,KBP1000J,South African Reserve Bank liabilities: Notes ...,1964,1964-01-01,355.0,355.0,MB,RMILL,A,Money and Banking,RMILL,Annual


<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [3]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [4]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce').astype('float64')
python_df['value'] = python_df['value'].astype('float64')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [5]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','value']] sliced from stata_df and python_df are exactly the same.

In [6]:
stata_sub = stata_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
stata_sub.equals(python_sub)

True